In [290]:
#import dependencies 
from numpy.random import seed
from numpy.random import randn
from scipy import stats
import numpy as np

import math  

import seaborn as sns
sns.set(style="darkgrid")
from matplotlib import pyplot as plt

import pandas as pd
import os

In [291]:
#check list of files in data directory 
print(os.listdir("./data"))

['newdata_iperf_m5d_48 - newdata_iperf_m5d_48.csv', 'ycruncher_m5d_aug13_48-instances - ycruncher_m5d_aug13.csv', '.DS_Store', 'm5d-no-idle-dedicated-host-11-07-2019 - newdata.csv', 'm5d_48vms_pgbench_2ndtry - m5d_48vms_pgbench_2ndtry.csv', 'm5d-y-cruncher-idle-mode-10-27-2019 - m5d-ycruncher-p-10-27-2019.csv-Summary.csv', 'm5d-sysbench-dedicated-host-10-24-2019 - m5d-sysbench-dedicated-host-10-24-2019.csv', 'm5d-no-idle-dedicated-host-11-07-2019 - newdata.csv-Summary.csv', 'y_cruncher_08_22_2019 - y_cruncher_08_22_2019.csv', 'm5d_48vms_pgbench_no_idle_11_11_2019 - Sheet1.csv', 'm5d-y-cruncher-idle-mode-10-27-2019 - m5d-ycruncher-p-10-27-2019.csv', 'summary']


In [292]:
#make sure to import correct file here
data_name = 'm5d_48vms_pgbench_no_idle_11_11_2019 - Sheet1.csv'

In [293]:
#adjust header if necessary
df_data = pd.read_csv("./data/" + data_name, header = 1)

In [294]:
#remove idle vm rows

#y-cruncher
#df_data.dropna(subset=['processorInfo'], inplace= True)

In [295]:
df_data.head()

,instanceID,experimentID,setId,instanceType,wallTime,clients,threads,scaleFactor,transactionsType,queryMode,duration,transactions,mountPoint
0,i-00201d4dd6e17aac2,1573493686-0,0,m5d.large,60.412139,10,10,1,TPC-B (sort of),simple,60 s,124782,data_directory = '/var/lib/postgresql/9.5/main'
1,i-00201d4dd6e17aac2,1573493686-0,0,m5d.large,60.346640,10,10,1,TPC-B (sort of),simple,60 s,124280,data_directory = '/var/lib/postgresql/9.5/main'
2,i-00201d4dd6e17aac2,1573493686-0,0,m5d.large,60.321836,10,10,1,TPC-B (sort of),simple,60 s,124011,data_directory = '/var/lib/postgresql/9.5/main'
3,i-002280d80dc30f869,1573493686-0,0,m5d.large,60.302091,10,10,1,TPC-B (sort of),simple,60 s,125381,data_directory = '/var/lib/postgresql/9.5/main'
4,i-002280d80dc30f869,1573493686-0,0,m5d.large,60.259652,10,10,1,TPC-B (sort of),simple,60 s,124771,data_directory = '/var/lib/postgresql/9.5/main'


In [314]:
#setup cols, metric for the relative experiment type
def setUpMetrics(experimentName):
    
    if experimentName == 'pgbench':
        cols = ['instanceID', 'setId', 'transactions']
        metric = 'transactions'
    elif experimentName == 'sysbench' or experimentName == 'y-cruncher':
        cols = ['instanceID','setId','wallTime']
        metric = 'wallTime'
    elif experimentName == 'iperf':
        df_data['Total'] = df_data['Upload'] + df_data['Download']
        cols = ['instanceID','setId','Upload', 'Download', 'Total']
        metric = 'Total'
    else:
        print('Illegal Experiment Name..')
    return metric, cols

In [315]:
#setup vars for each experiment time... i.e. wallTime vs transactions
experimentName = 'pgbench'
metric, cols = setUpMetrics(experimentName)

In [297]:
#drop other columns
df_data = df_data[cols]

In [298]:
df_data.head()

,instanceID,setId,transactions
0,i-00201d4dd6e17aac2,0,124782
1,i-00201d4dd6e17aac2,0,124280
2,i-00201d4dd6e17aac2,0,124011
3,i-002280d80dc30f869,0,125381
4,i-002280d80dc30f869,0,124771


In [299]:
df_data.describe()

,setId,transactions
count,3528.000000,3528.000000
mean,15.666667,144260.246882
std,11.427711,16052.733702
min,0.000000,113248.000000
25%,6.000000,129251.000000
50%,14.000000,138340.500000
75%,24.000000,155062.500000
max,47.000000,186691.000000


In [320]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3528 entries, 0 to 3527
Data columns (total 3 columns):
instanceID      3528 non-null object
setId           3528 non-null int64
transactions    3528 non-null int64
dtypes: int64(2), object(1)
memory usage: 82.8+ KB


In [347]:
#group by tenants / setId 
mean = df_data.groupby('setId').mean()
mean.head()

,transactions
setId,
0,124247.416667
1,124609.000000
2,125635.673913
3,127079.318519
4,129004.856061


In [348]:
#create result dataframe
result_df = mean

In [349]:
#create std, normalized data
std = df_data.groupby('setId').std()
std.head()

,transactions
setId,
0,1914.137745
1,1923.470182
2,2196.456487
3,2707.895382
4,4501.143763


In [350]:
#add normalized column
if experimentName == 'pgbench':
    result_df['normalized'] = (result_df.loc[:,metric:]-result_df.iloc[-1][metric:]).div(result_df.iloc[-1][metric:])
    result_df['normalized'] *= 100
    result_df['normalized'] += 100
else: 
    result_df['normalized'] = 1 - (result_df.loc[:,metric:]-result_df.iloc[-1][metric:]).div(result_df.iloc[-1][metric:])
    result_df['normalized'] *= 100

result_df['normalized'] = result_df['normalized'].round(2).astype(str) + '%'

In [351]:
result_df.head()

,transactions,normalized
setId,,
0,124247.416667,66.97%
1,124609.000000,67.17%
2,125635.673913,67.72%
3,127079.318519,68.5%
4,129004.856061,69.53%
5,129609.550388,69.86%
6,130127.285714,70.14%
7,131178.869919,70.71%
8,132259.575000,71.29%


In [352]:
#add std column
result_df['std'] = std

In [353]:
#reverse table
result_df=result_df.iloc[::-1]

In [354]:
#add tenants
result_df['tenants'] = np.arange(len(result_df))+1

/Users/dell/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [355]:
#rearrange columns
cols = ['tenants', metric, 'std', 'normalized']

In [356]:
result_df = result_df[cols]

In [357]:
#add % to normalized

In [358]:
result_df

,tenants,transactions,std,normalized
setId,,,,
47,1,185525.666667,277.831484,100.0%
46,2,182739.333333,2596.633949,98.5%
45,3,183869.444444,2700.515280,99.11%
44,4,183794.000000,2225.317056,99.07%
43,5,179218.866667,3471.788812,96.6%
42,6,177914.444444,4026.656213,95.9%
41,7,173635.095238,6399.124768,93.59%
40,8,173138.333333,6106.710426,93.32%
39,9,170995.000000,3180.109251,92.17%


In [289]:
#print to csv
result_df.to_csv( path_or_buf='./data/summary/' + data_name + '-Summary.csv')

In [ ]:
# graph tenants vs wallTime